In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px
from LocalPath import LOCALPATH
from DBSCANSupport import DBSCANSupport

In [3]:
full_curve = pd.read_csv(LOCALPATH + 'data/curv_32.highpass.xyz', sep='\t', header=None, names=['x', 'y', 'z'])
test_zone = full_curve[(full_curve['x']  > -100) & (full_curve['x'] < -80) & (full_curve['y'] > -10) & (full_curve['y'] < 0)]
test_zone.to_csv(LOCALPATH + 'data/test_curv_32.highpass.xyz', index=False)